# Assignment 59, Uniformly sampling SO(3)—2

Here, rather than starting with unit quaternions and converting to matrices, we instead randomly sample unit vectors and angles. From these, we create the corresponding roation matrices, making free use of the fact that the rotation matrix $R_{(\hat{u}, \theta)}$ giving the rotation by angle $\theta$ about the line containing the unit vector $\hat{u}$ can be given by 

$$R_{(\hat{u}, \theta)} = \cos(\theta) \cdot \mathbf{I}_{3 \times 3} + (\sin(\theta))\cdot[\hat{u}]_{\times} + (1 - \cos(\theta)\cdot (\hat{u} \otimes \hat{u})$$

where $[\hat{u}]_{\times}$ is given by

$$[\hat{u}]_{\times} = \sum_{i = 1}^{3}(\hat{u} \times \hat{e}_{i}) \otimes \hat{e}_{i} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

np.random.seed(42)

# compute the cross - product matrix for a unit vector u
def crossProductMatrix(u):
    """
        u: a 3 compnent numpy array representing
           a (normalized) vector 
               
        returns: a 3 x 3 matrix representing the 
                 cross product matrix of u
    """
    e1 = np.array([1.0, 0.0, 0.0])
    e2 = np.array([0.0, 1.0, 0.0])
    e3 = np.array([0.0, 0.0, 1.0])
    e = [e1, e2, e3]
    m = np.eye(3)
    for i in range(3):
        m += np.outer(np.cross(u, e[i]), e[i])
    return m

def rotationMatrix(theta, u):
    ux = crossProductMatrix(u)
    return (np.cos(theta) * np.eye(3)) + (np.sin(theta) * ux) + ((1.0 - np.cos(theta)) * np.outer(u, u))

N = 10000

vectors = np.array([np.random. normal(0, 1, 3) for i in range(N)])
unit_vectors = np.array( [(1.0 / np.sqrt(np.dot(v, v))) * v for v in vectors] )
angles = np.random.uniform(0, np.pi,  N)    

M = list(map(rotationMatrix, angles, unit_vectors))

# define a function on the matrix representations
# this is the sum over the abs values of the matrix elements
f = lambda M: np.sum(np.abs(M.flatten()))

# pick one of the matrices at random to be R0
R0 = M[int(np.random.randint(low=0, high=100, size=1))]

f_R_values = list(map(f, M))

f_R0_R_values = list(map(f, list(map(lambda R: R0 @ R, M))))

def plot_hist(z, N):
    plt.hist(z, density = True, bins = int(np.sqrt(N)))
    plt.show()
    plt.close()

In [ ]:
plot_hist(f_R_values, N)

In [ ]:
plot_hist(f_R0_R_values, N)

In [ ]:
ks_2samp(f_R_values, f_R0_R_values)

## Conclusions

The manner in which the random matrices are generated obviously has an impact on the shape of the resulting distributions. The distributions obtained here are both roughly bimodal, and don't shoe the same pronounced skew that the distributions in the previous problem did. Note that we are mixing distributions in generating the matrices using axis and angle sampling as done above, since the unit vectors are chosen using a normal distribution and the angles are chosen using a uniform distribution. The bottom distribution is also shifted slightly relative to the top distribution, another difference with the former sampling methods. The value of the test statistic is over 10 times larger than it was using the prior sampling method, though the p - value we obtain here is indistinguishable from zero. The evidence that the two distributions arise from the same undelying density is thus here slighlty less strong than in the former case, but is still relatively string overall. 